In [9]:
from typing import List

## Radix Sort

### Overview

> The idea of the Radix Sort is to upgrade Counting Sort so that it **maintains the linear time complexity even if the range of elements' values drastically exceeds** the number of elements.
>
> In fact, Radix Sort inherently uses Counting Sort as the main subroutine

**Principle idea**: 
 
 Radix sort avoids comparison by creating and distributing elements into buckets **according to their radix**. 
 
 For elements with more than one significant digit, this bucketing process is repeated for each digit, **while preserving the ordering of the prior step**, until all digits have been considered.


#### Digit Order

Radix sorts can be implemented to start at either the most significant digit (MSD) or least significant digit (LSD). For example, with 1234, one could start with 1 (MSD) or 4 (LSD).

LSD radix sorts typically use the following sorting order: short keys come before longer keys, and then keys of the same length are sorted lexicographically. **This coincides with the normal order of integer representations**, like the sequence **[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]**.<br/>
(LSD sorts are generally <u>stable sorts</u>)

MSD radix sorts are most suitable for **sorting strings or fixed-length integer representations**. A sequence like **[b, c, e, d, f, g, ba]** would be sorted as **[b, ba, c, d, e, f, g]**. If lexicographic ordering is used to sort variable-length integers in base 10, then numbers from 1 to 10 would be output as **[1, 10, 2, 3, 4, 5, 6, 7, 8, 9]**, as if the shorter keys were left-justified and padded on the right with blank characters to make the shorter keys as long as the longest key. <br/>
(MSD sorts are not necessarily stable if the original ordering of duplicate keys must always be maintained)

Other than the traversal order, MSD and LSD sorts differ in their handling of variable length input:
- LSD sorts can group by length, radix sort each group, then concatenate the groups in size order. 
- MSD sorts must effectively 'extend' all shorter keys to the size of the largest key and sort them accordingly, which can be more complicated than the grouping required by LSD.

However, **MSD sorts are more amenable to subdivision and recursion**. Each bucket created by an MSD step can itself be radix sorted using the next most significant digit, without reference to any other buckets created in the previous step. Once the last digit is reached, concatenating the buckets is all that is required to complete the sort. 

### Examples

#### Least significant digit



Input list:

**[170, 45, 75, 90, 2, 802, 2, 66]**

Starting from the rightmost (last) digit, sort the numbers based on that digit:

**[{17<u>0</u>, 9<u>0</u>}, {<u>2</u>, 80<u>2</u>, <u>2</u>}, {4<u>5</u>, 7<u>5</u>}, {6<u>6</u>}]**

Sorting by the next left digit:

**[{02, 802, 02}, {45}, {66}, {170, 75}, {90}]**

>    Notice that an implicit digit 0 is prepended for the two 2s so that 802 maintains its position between them.

And finally by the leftmost digit:

**[{002, 002, 045, 066, 075, 090}, {170}, {802}]**

>    Notice that a 0 is prepended to all of the 1- or 2-digit numbers.
>
Each step requires just a single pass over the data, since each item can be placed in its bucket without comparison with any other element. 


#### Most significant digit

Input list, fixed width numeric strings with leading zeros:

**[170, 045, 075, 025, 002, 024, 802, 066]**

First digit, with brackets indicating buckets:

**[{<u>0</u>45, <u>0</u>75, <u>0</u>25, <u>0</u>02, <u>0</u>24, <u>0</u>66}, {<u>1</u>70}, {<u>8</u>02}]**

 > Notice that 170 and 802 are already complete <font color=orange>because they are all that remain in their buckets</font>, so no further recursion is needed

Next digit:

**[{ {002}, {025, 024}, {045}, {066}, {075} }, 170, 802]**

Final digit:

**[ 002, { {024}, {025} }, 045, 066, 075 , 170, 802]**

All that remains is concatenation:

**[002, 024, 025, 045, 066, 075, 170, 802]**

### Implementation

In [7]:
def intlen(x: int) -> int: return len(str(x))

# A utility function to get the digit at index d in a integer
def digit_at(x, d):
    return int(x / (10**(d-1))) % 10

def getRadix(x, pos: int) -> int:
    if pos > intlen(x):
        return 0
    return int(str(x)[-pos])

#### LSD

In [23]:
def radix_sort_LSD(arr: List[int]) -> List[int]:
    max_digits = intlen(max(arr))
    for pos in range(1, max_digits + 1):
        buckets = [[] for _ in range(10)]
        for i in range(len(arr)):
            radix = getRadix(arr[i], pos)
            buckets[radix].append(arr[i])
        arr = [item for sublist in buckets for item in sublist]
    return arr

In [26]:
arr = [741, 8586, 5710, 170, 45, 75, 90, 2, 718, 8977, 6, 802, 2, 66, 790, 95, 9807]
sorted_arr_LSD = radix_sort_LSD(arr)
sorted_arr_LSD

[2, 2, 6, 45, 66, 75, 90, 95, 170, 718, 741, 790, 802, 5710, 8586, 8977, 9807]

#### MSD

##### Using Linked List

In [ ]:
import math



##### Using Counting Sort

In [19]:
def radix_sort_MSD(arr: List[int], lo: int, hi: int, d: int) -> List[int]:
    '''
    Most significant digits radix sort
    
    Args:
        arr (list[int]): the array to be sorted
        lo (int): the starting index, which is 0 at first and then the starting index of a bucket during recursion
        hi (int): the ending index, which is len(arr) - 1 at first and then the ending index of a bucket during recursion
        d (int): the position of the radix, between 1 and the length of maximum value in the array  
    '''
    # if the counting bucket is empty or contains only one element 
    # end this recursion (move to the next digit and remove this only element out of buckets if there's an element)
    if lo >= (hi - 1): return
    count = [0] * 10
    # temp is used to easily swap the elements in the original arr, instead of swap directly in the original arr, which will disrupt the origin order  
    tmp = [0 for _ in range(len(arr))]
    # Store the occurrences of radixes
    for i in range(lo, hi):
        count[digit_at(arr[i], d)] += 1
    # Change count[] so that count[] contains the actual positions of the radixes
    for i in range(1, len(count)):
        count[i] += count[i - 1]
    # build the temp, which contains the relatively ordered elements
    for i in range(hi - 1, lo - 1, -1):
        radix = digit_at(arr[i], d)
        count[radix] -= 1
        tmp[count[radix]] = arr[i]
    # Put ordered elements back to the array, so that the arr now contains partially ordered elements
    for i in range(lo, hi):
        arr[i] = tmp[i - lo] # !important, minus `lo`, because the left removed elements are not counted any more
    # Recursively MSD_sort() on each partially sorted buckets to sort them by their next digit
    for r in range(len(count) - 1):
        radix_sort_MSD(arr, lo + count[r], lo + count[r + 1], d - 1)
    return arr
    

In [27]:
arr = [741, 8586, 5710, 170, 45, 75, 90, 2, 718, 8977, 6, 802, 2, 66, 790, 95, 9807]
sorted_arr_MSD = radix_sort_MSD(arr, 0, len(arr), intlen(max(arr)))
sorted_arr_MSD

[2, 2, 6, 45, 66, 75, 90, 95, 170, 718, 741, 790, 802, 5710, 8586, 8977, 9807]

### Analysis

#### Complexity Analysis

- Time Complexity:
    - LSD Radix sort: Best and Worst-Case time complexity is $O(N*M)$ where $M$ = length of the longest string.
    - MSD Radix sort: Best Case time complexity is $O(N)$ and the Worst-Case time complexity is $O(N*M)$ where $M$ = the average length of strings.
- Auxiliary Space:
    - LSD Radix sort: $O(N + B)$
    - MSD Radix sort: $O(N + MB)$, where M = length of the longest string and $B$ = size of radix ($B=10$ possible numbers or $B=256$ characters or $B=2$ for Binary).

MSD uses recursion, so it requires more space than LSD. This means that MSD is much slower than LSD when working with a few inputs.

#### Characteristics

 **non-comparative** sorting algorithm:  rather uses the inherent nature of the data its sorting to sort faster - it sorts data based on their radix.
 > Comparative sorting algorithms have the best case time complexity of $O(nlogn)$, which is comparatively worse to linear execution time $(O(n+k))$ of non-comparative algorithms.